In [1]:
import os
import pickle
import torch
import numpy as np
from math import ceil
from model_vc import Generator


def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

device = 'cuda:0'
G = Generator(32,256,512,32).eval().to(device) # 2nd number is 

g_checkpoint = torch.load('autovc.ckpt' ,map_location='cuda:0')
G.load_state_dict(g_checkpoint['model'])

metadata = pickle.load(open('metadata.pkl', "rb"))

spect_vc = []
      

In [5]:

for sbmt_i in metadata:
             
    x_org = sbmt_i[2]
    x_org, len_pad = pad_seq(x_org)
    uttr_org = torch.from_numpy(x_org[np.newaxis, :, :]).to(device)
    emb_org = torch.from_numpy(sbmt_i[1][np.newaxis, :]).to(device)
    
    for sbmt_j in metadata:
                   
        emb_trg = torch.from_numpy(sbmt_j[1][np.newaxis, :]).to(device)
        
        with torch.no_grad():
            _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
            
        if len_pad == 0:
            uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
        else:
            uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()
        
        spect_vc.append( ('{}x{}'.format(sbmt_i[0], sbmt_j[0]), uttr_trg) )
        
        
with open('results.pkl', 'wb') as handle:
    pickle.dump(spect_vc, handle)    

In [2]:
metadata

[['p225',
  array([-2.92885415e-02,  1.67739280e-02,  6.42375797e-02,  5.12384847e-02,
          8.77934247e-02, -4.05867286e-02,  1.08849108e-02,  2.06673276e-02,
         -8.39690343e-02,  1.48199843e-02,  3.82344425e-02,  1.12243919e-02,
          4.62971367e-02,  1.66661311e-02, -5.78785129e-02,  3.60288732e-02,
          1.92339886e-02,  9.17971320e-03,  2.74707917e-02, -5.48039749e-02,
          2.50798557e-02,  4.66737375e-02, -6.14981353e-03,  3.88026945e-02,
          5.68139665e-02, -7.33052716e-02,  2.30920967e-02, -1.04292825e-01,
         -2.61898227e-02,  3.02257240e-02, -3.02889403e-02,  3.63447554e-02,
         -4.97230627e-02,  1.57715172e-01, -3.93295921e-02,  5.51161245e-02,
          4.64604087e-02, -4.59927395e-02, -4.82378080e-02, -3.81431282e-02,
          3.91379185e-02,  4.59317304e-02, -1.55072343e-02, -1.67513415e-02,
         -5.09507731e-02, -5.08496165e-02, -2.25679334e-02,  2.63878461e-02,
         -1.26611767e-02, -2.49883570e-02,  3.02621610e-02, -2.484

In [4]:
metadata = metadata[:2]

In [13]:
for i in metadata[0][1:]:
    print(i.shape)

(256,)
(90, 80)


In [6]:
type(spect_vc)

list

In [7]:
spect_vc

[('p225xp225',
  array([[0.38280222, 0.32192808, 0.27241027, ..., 0.18827905, 0.18116055,
          0.1747563 ],
         [0.3774055 , 0.30501315, 0.3142484 , ..., 0.23313922, 0.18586272,
          0.14988673],
         [0.38855177, 0.28357574, 0.2739416 , ..., 0.13206424, 0.10279437,
          0.10369422],
         ...,
         [0.57143515, 0.6357634 , 0.5616601 , ..., 0.21222197, 0.19688365,
          0.182007  ],
         [0.5051674 , 0.50225836, 0.48195556, ..., 0.18236066, 0.16847168,
          0.15469953],
         [0.44893792, 0.44055843, 0.4609493 , ..., 0.14624406, 0.1329227 ,
          0.13138215]], dtype=float32)),
 ('p225xp228',
  array([[0.4372439 , 0.35739934, 0.30414355, ..., 0.20524159, 0.19167477,
          0.17206734],
         [0.44479764, 0.35146436, 0.31340063, ..., 0.23535405, 0.23473123,
          0.22268619],
         [0.4352065 , 0.36368087, 0.34444585, ..., 0.2080973 , 0.19319978,
          0.18029557],
         ...,
         [0.4788336 , 0.4589783 , 0.497278

In [16]:
spect_vc[0][1].shape

(90, 80)

In [2]:

with open('results.pkl', 'rb') as handle:
    results = pickle.load(handle)    

In [3]:
results

[('p225xp225',
  array([[0.38280213, 0.32192802, 0.27241027, ..., 0.18827908, 0.18116057,
          0.1747563 ],
         [0.3774054 , 0.3050131 , 0.31424835, ..., 0.23313923, 0.1858627 ,
          0.1498867 ],
         [0.3885517 , 0.2835757 , 0.27394158, ..., 0.13206425, 0.10279435,
          0.10369419],
         ...,
         [0.57143515, 0.63576335, 0.5616601 , ..., 0.21222192, 0.19688356,
          0.18200693],
         [0.50516737, 0.50225824, 0.48195547, ..., 0.18236066, 0.16847159,
          0.15469947],
         [0.44893786, 0.4405584 , 0.46094933, ..., 0.14624406, 0.13292266,
          0.13138218]], dtype=float32)),
 ('p225xp228',
  array([[0.43724394, 0.3573994 , 0.30414352, ..., 0.2052415 , 0.19167465,
          0.17206722],
         [0.4447977 , 0.35146436, 0.31340063, ..., 0.23535405, 0.23473118,
          0.22268617],
         [0.4352065 , 0.3636808 , 0.34444585, ..., 0.20809734, 0.19319983,
          0.18029559],
         ...,
         [0.4788337 , 0.4589783 , 0.497278